In [3]:
!wget https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.13/24.08.1/rapids-4-spark_2.13-24.08.1-cuda12.jar

--2024-09-27 16:15:55--  https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.13/24.08.1/rapids-4-spark_2.13-24.08.1-cuda12.jar
Resolving repo1.maven.org (repo1.maven.org)... 146.75.92.209, 2a04:4e42:7a::209
Connecting to repo1.maven.org (repo1.maven.org)|146.75.92.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500547780 (477M) [application/java-archive]
Saving to: ‘rapids-4-spark_2.13-24.08.1-cuda12.jar’

k_2.13-24.08.1-cuda  70%[=============>      ] 336.47M  21.6MB/s    eta 7s     

rapids-4-spark_2.13 100%[===================>] 477.36M  21.5MB/s    in 23s     

2024-09-27 16:16:43 (20.4 MB/s) - ‘rapids-4-spark_2.13-24.08.1-cuda12.jar’ saved [500547780/500547780]



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, isnan, isnull
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import Word2Vec
from pyspark.ml.classification import RandomForestClassifier

spark = SparkSession.builder.appName(
            "SentimentAnalysisBaseline"
        ).config(
            "spark.executor.memory", "16g"
        ).config(
            "spark.driver.memory", "32g"
        ).config(
            "spark.executor.extraClassPath", "rapids-4-spark_2.13-24.08.1-cuda12.jar"
        ).config(
            "spark.driver.extraClassPath", "rapids-4-spark_2.13-24.08.1-cuda12.jar"
        ).getOrCreate()

df = spark.read.csv("./ggg_sg.csv", header=True, inferSchema=True, multiLine=True, escape='"')
df = df.filter(df.ContextualText.isNotNull())
df = df.filter(df.DocTone.isNotNull())
df = df.withColumn("DocTone", df["DocTone"].cast(FloatType()))
# Create sentiment label: Positive (2), Neutral (1), Negative (0)
def sentiment_label(score):
    if score > 1.9910:
        return 2
    elif score < -2.0202:
        return 0
    else:
        return 1

sentiment_udf = udf(sentiment_label, IntegerType())

df = df.withColumn("label", sentiment_udf(col("DocTone")))
tokenizer = Tokenizer(inputCol="ContextualText", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Feature extraction (Word2Vec)
word2Vec = Word2Vec(vectorSize=100, minCount=5, inputCol="filtered_words", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, remover, word2Vec])
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed=42)

pipelineModel = pipeline.fit(trainingData)
trainingData = pipelineModel.transform(trainingData)
testData = pipelineModel.transform(testData)

# Random Forest model
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=50, maxDepth=10, seed=42)

rfModel = rf.fit(trainingData)

In [10]:
df = df.filter(df.ContextualText.isNotNull())
df = df.filter(df.DocTone.isNotNull())
df = df.withColumn("DocTone", df["DocTone"].cast(FloatType()))
# Create sentiment label: Positive (2), Neutral (1), Negative (0)
def sentiment_label(score):
    if score > 1.9910:
        return 2
    elif score < -2.0202:
        return 0
    else:
        return 1

sentiment_udf = udf(sentiment_label, IntegerType())

df = df.withColumn("label", sentiment_udf(col("DocTone")))
tokenizer = Tokenizer(inputCol="ContextualText", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Feature extraction (Word2Vec)
word2Vec = Word2Vec(vectorSize=100, minCount=5, inputCol="filtered_words", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, remover, word2Vec])
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed=42)

pipelineModel = pipeline.fit(trainingData)
trainingData = pipelineModel.transform(trainingData)
testData = pipelineModel.transform(testData)

# Random Forest model
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=50, maxDepth=10, seed=42)

rfModel = rf.fit(trainingData)

24/09/27 18:31:50 ERROR TaskContextImpl: Error in TaskCompletionListener
org.apache.spark.SparkException: Block broadcast_6 does not exist
	at org.apache.spark.errors.SparkCoreErrors$.blockDoesNotExistError(SparkCoreErrors.scala:318)
	at org.apache.spark.storage.BlockInfoManager.blockInfo(BlockInfoManager.scala:269)
	at org.apache.spark.storage.BlockInfoManager.unlock(BlockInfoManager.scala:390)
	at org.apache.spark.storage.BlockManager.releaseLock(BlockManager.scala:1309)
	at org.apache.spark.broadcast.TorrentBroadcast.$anonfun$releaseBlockManagerLock$1(TorrentBroadcast.scala:319)
	at org.apache.spark.broadcast.TorrentBroadcast.$anonfun$releaseBlockManagerLock$1$adapted(TorrentBroadcast.scala:319)
	at org.apache.spark.TaskContext$$anon$1.onTaskCompletion(TaskContext.scala:137)
	at org.apache.spark.TaskContextImpl.$anonfun$invokeTaskCompletionListeners$1(TaskContextImpl.scala:144)
	at org.apache.spark.TaskContextImpl.$anonfun$invokeTaskCompletionListeners$1$adapted(TaskContextImpl.scal

In [ ]:
predictions = rfModel.transform(testData)
# Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % accuracy)

# Detailed evaluation
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)

evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluator_f1.evaluate(predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

predictions.select("ContextualText", "label", "prediction").show(10, truncate=50)